In [8]:
%%capture
!pip install hummingbird-ml

In [9]:
import pickle
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
with open('drive/MyDrive/RealLifeViolencePickle/output.pickle', 'rb') as handle:
    output = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/labels_output.pickle', 'rb') as labels:
    labels_output = pickle.load(labels)

In [12]:
from hummingbird.ml import convert,load
model = load('drive/MyDrive/EventDetection/saved_models/pytorch_decisiontree')
predicted_labels = model.predict(output)
print(predicted_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1]


In [13]:
videoPath = 'drive/MyDrive/EventDetection/real_violence_kaggle/Test/Violent/V_8.mp4'

In [41]:
from google.colab.patches import cv2_imshow
# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(videoPath)
writer = None
(W, H) = (None, None)
count = -1
# loop over frames from the video file stream
while count < 27:
		# read the next frame from the file
		(grabbed, frame) = vs.read()
		count += 1
		# if the frame was not grabbed, then we have reached the end
		# of the stream
		if not grabbed:
			break
		# if the frame dimensions are empty, grab them
		if W is None or H is None:
			(H, W) = frame.shape[:2]
		# clone the output frame, then convert it from BGR to RGB
		# ordering, resize the frame to a fixed 224x224, and then
		# perform mean subtraction
		output = frame.copy()
		frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		frame = cv2.resize(frame, (224, 224)).astype("float32")
		# make predictions on the frame and then update the predictions
		# queue
		if predicted_labels[count] == 1:
			label = "Violent"
			color = (0, 0, 255)
		else:
			label = "Non-Violent"
			color = (0, 255, 0)
		# draw the activity on the output frame
		text = "activity: {}".format(label)
		cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
			1.25, color, 5)
		# check if the video writer is None
		if writer is None:
			# initialize our video writer
			fourcc = cv2.VideoWriter_fourcc(*"MJPG")
			writer = cv2.VideoWriter('drive/MyDrive/EventDetection/output.avi', fourcc, 30,
				(W, H), True)
		# write the output frame to disk
		writer.write(output)
		key = cv2.waitKey(1) & 0xFF
		# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...


In [42]:
!ffmpeg -i drive/MyDrive/EventDetection/output.avi drive/MyDrive/EventDetection/output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [43]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "drive/MyDrive/EventDetection/output.mp4"

# Compressed video path
# compressed_path = "/content/videos/result_compressed.mp4"

# os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(save_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)